# Overview Exercises
### For each of the following questions, formulate a null and alternative hypothesis (be as specific as you can be), then give an example of what a true positive, true negative, type I and type II errors would look like. Note that some of the questions are intentionally phrased in a vague way. It is your job to reword these as more precise questions that could be tested.

### Has the network latency gone up since we switched internet service providers?

##### null hypothesis

##### alternative hypothesis

##### true positive

##### true negative

##### type I error

##### type II error

### Is the website redesign any good?

##### null hypothesis

##### alternative hypothesis

##### true positive

##### true negative

##### type I error

##### type II error

### Is our television ad driving more sales?

##### null hypothesis

##### alternative hypothesis

##### true positive

##### true negative

##### type I error

##### type II error

# T-Test Exercises

### Ace Realty wants to determine whether the average time it takes to sell homes is different for its two offices. A sample of 40 sales from office #1 revealed a mean of 90 days and a standard deviation of 15 days. A sample of 50 sales from office #2 revealed a mean of 100 days and a standard deviation of 20 days. Use a .05 level of significance.

### Load the mpg dataset and use it to answer the following questions:

##### Is there a difference in fuel-efficiency in cars from 2008 vs 1999?

##### Are compact cars more fuel-efficient than the average car?

##### Do manual cars get better gas mileage than automatic cars?